<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Rubén Hernández Guevara | Bryan Azahel Juárez Pineda.

**Fecha:** 25 de Abril del 2021.

**Expediente** : if717710 | if722176.
**Profesor:** Oscar David Jaramillo Zuluaga.

# Tarea 9: Clase 22

## Enunciado de tarea (Transcibirlo)
> Replicar el procedimiento anterior para valoración de opciones 'call', pero en este caso para opciones tipo 'put' y compararlo con el valor teórico de la ecuación de Black-Scholes

# Solución Bryan

In [ ]:
# Código de solución 
.
.
.
.


# Solución Rubén

In [ ]:
# Código de solución 
.
.
.
.
